## Simulation comparing naive GLMMLasso and Selective Inference selecting lambda from the data
Comparing TPR and FDR for the two methods in different settings of SNR and proportion of non-zero coefficients.
We select the appropriate value of lambda from the whole data, and use BH procedure to set the FDR control level at 10%.

In [ ]:
n_subjects= 25
n_observations = 4
p = 100
q = 0

SNR_values = c(2,4)
prop_relevant_values = c(0.05, 0.1, 0.2)
num_iter = 50

fixed_form = as.formula(
    paste("y ~ 1 +",paste("X", 1:p, sep="", collapse='+'),sep="")
)

rand_form = list(subjects =~ 1)

tot_its <- length(SNR_values)*length(prop_relevant_values)*num_iter

columns = c('SNR','prop_relevant', 'TPR_lasso', 'TPR_selinf', 'FDR_lasso', 'FDR_selinf', 'num_sel_lasso', 'num_sel_selinf')
results_df = data.frame(matrix(nrow = 0, ncol = length(columns))) 
colnames(results_df) = columns

pb = txtProgressBar(min = 0, max = tot_its, initial = 0, style = 3)

for(SNR in SNR_values){
    for(prop_relevant in prop_relevant_values){
        for(i in 1:num_iter){

            data <- data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant, rho=0.5)
            X <- data$X
            subjects <- data$subjects
            y <- data$y
            beta <- data$beta

            sel <- selFun_adapting_lambda(X, subjects, y, fixed_form, rand_form)
            sel_vec <- sel$vec
            sel_names <- sel$names
            lambda <- sel$lambda

            selFun <- function(y) selFun_fixed_lambda(X, subjects, y, fixed_form, rand_form, lambda)

            # Now we can define the function checking the congruency
            # with the original selection
            checkFun <- function(yb){

                all(selFun(yb)$vec == sel_vec)

            }

            sel_form = as.formula(
            paste("y ~ ",paste(sel_names[2:length(sel_names)], collapse='+'), "+ (1|subjects)")
            )

            final_model = lmer(formula = sel_form, data = data.frame(X, subjects, y))
            
            # and compute valid p-values conditional on the selection
            # (this takes some time and will produce a lot of warnings)
            suppressWarnings(res <- mocasin(final_model, this_y = y, conditional = FALSE,
                        checkFun = checkFun, nrSamples = 100, trace = FALSE))

            sel_with_selinf <- selection_with_selinf(res, sel_vec, fdr_level = 0.1)
            
            lasso_metrics <- metrics(sel_vec,c(1,beta!=0))
            selinf_metrics <- metrics(sel_with_selinf,c(1,beta!=0))

            num_sel_lasso <- sum(sel_vec)
            num_sel_selinf <- sum(sel_with_selinf)

            results_df[nrow(results_df) + 1,] <- c(SNR, prop_relevant, lasso_metrics$tpr, selinf_metrics$tpr, lasso_metrics$fdr, selinf_metrics$fdr, num_sel_lasso, num_sel_selinf)

            setTxtProgressBar(pb,nrow(results_df))

        }
    }
}

close(pb)

In [13]:
write.csv(results_df, "C:/Users/dalma/Desktop/Matteo/phd/r/simulation_results_sel_lambda.csv", row.names=FALSE)